# VARI - TensorFlow Single-Output Training

This notebook trains a single-output **TensorFlow / Keras** CNN to predict **avg_VARI** from preprocessed RGB images.

**Paths used (change if needed):**
- `csv_file = '/content/image_avg_indices.csv'`
- `preprocessed_dir = '/content/visualization/outlier'`

The trained model will be saved as **/mnt/data/vari_model.h5** (HDF5) and as a SavedModel folder **/mnt/data/vari_saved_model**.

> Note: TensorFlow models are typically saved as `.h5` or SavedModel; `.pkl` is not the standard format for TF models. If you specifically need a `.pkl`, I can export model weights or configuration into a pickle-friendly format on request.

In [1]:
# Imports and device check
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import joblib

print('TensorFlow version:', tf.__version__)
print('GPU available:', tf.config.list_physical_devices('GPU'))

2025-09-14 13:45:56.861450: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-14 13:45:56.861655: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-14 13:45:56.892874: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-14 13:45:57.682821: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

TensorFlow version: 2.20.0
GPU available: []


E0000 00:00:1757837758.397976   19088 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1757837758.408247   19088 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
# Helper: read CSV and collect paths & labels
IMG_SIZE = 224

def make_dataset_from_csv(csv_file, preprocessed_dir, target_column):
    df = pd.read_csv(csv_file)
    image_paths = []
    labels = []
    missing = []
    for _, row in df.iterrows():
        img_name = row['image_id']
        pre_name = img_name.replace('.png', '_outlier_removed.png')
        path = os.path.join(preprocessed_dir, pre_name)
        if os.path.exists(path):
            image_paths.append(path)
            labels.append(float(row[target_column]))
        else:
            missing.append(path)
    print(f'Found {len(image_paths)} images for target {target_column} (missing {len(missing)})')

    return np.array(image_paths, dtype=str), np.array(labels, dtype=np.float32)


In [3]:
# Preprocessing functions for tf.data using pure TF ops
import tensorflow as tf

def preprocess_image_tf(path, label, augment=False):
    path = tf.cast(path, tf.string)
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)  # [0,1]
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])

    if augment:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_brightness(image, max_delta=0.05)
        image = tf.image.random_contrast(image, lower=0.95, upper=1.05)

    # normalize with ImageNet stats
    mean = tf.constant([0.485, 0.456, 0.406], dtype=tf.float32)
    std = tf.constant([0.229, 0.224, 0.225], dtype=tf.float32)
    image = (image - mean) / std

    return image, tf.expand_dims(label, axis=-1)

def make_tf_dataset(paths, labels, batch_size=16, shuffle=True, augment=False):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(paths), reshuffle_each_iteration=True)
    ds = ds.map(lambda p, l: preprocess_image_tf(p, l, augment), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds


In [4]:
# Build a simple CNN with an SE (Squeeze-and-Excitation) block for channel attention
from tensorflow.keras import layers, models

def se_block(input_tensor, reduction=8):
    filters = input_tensor.shape[-1]
    se = layers.GlobalAveragePooling2D()(input_tensor)
    se = layers.Dense(max(filters // reduction, 4), activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)
    se = layers.Reshape((1,1,filters))(se)
    return layers.multiply([input_tensor, se])


def build_model(input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D(2)(x)

    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D(2)(x)

    x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D(2)(x)

    x = layers.Conv2D(512, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    x = se_block(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(1, activation='linear')(x)

    model = models.Model(inputs, outputs)
    return model


In [5]:
# # Main training flow - replace paths at top if needed
# csv_file = '/content/image_avg_indices.csv'
# preprocessed_dir = '/content/visualization/outlier'
#
# if not os.path.exists(csv_file) or not os.path.exists(preprocessed_dir):
#     raise FileNotFoundError('CSV or preprocessed directory not found. Upload to the paths above or edit the variables in this cell.')
#
# paths, labels = make_dataset_from_csv(csv_file, preprocessed_dir, target_column='avg_VARI')
#
# # splits
# n = len(paths)
# train_n = int(0.7 * n)
# val_n = int(0.15 * n)
#
# np.random.seed(42)
# perm = np.random.permutation(n)
# train_idx = perm[:train_n]
# val_idx = perm[train_n:train_n+val_n]
# test_idx = perm[train_n+val_n:]
#
# train_paths, train_labels = paths[train_idx], labels[train_idx]
# val_paths, val_labels = paths[val_idx], labels[val_idx]
# test_paths, test_labels = paths[test_idx], labels[test_idx]
#
# print('Train/Val/Test sizes:', len(train_paths), len(val_paths), len(test_paths))
#
# # Create tf.data datasets
# batch_size = 16
# train_ds = make_tf_dataset(train_paths, train_labels, batch_size=batch_size, shuffle=True, augment=True)
# val_ds = make_tf_dataset(val_paths, val_labels, batch_size=batch_size, shuffle=False, augment=False)
# test_ds = make_tf_dataset(test_paths, test_labels, batch_size=batch_size, shuffle=False, augment=False)
#
# # build model
# model = build_model()
# model.compile(optimizer=optimizers.Adam(learning_rate=5e-4), loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
# model.summary()
#
# # callbacks
# es = callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
# rlrp = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=8, min_lr=1e-6)
#
# # train
# history = model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[es, rlrp])
#
# # evaluate
# eval_res = model.evaluate(test_ds)
# print('Test eval (loss, rmse):', eval_res)
#
# # save model
# model.save('/mnt/data/vari_model.h5')
# model.save('/mnt/data/vari_saved_model', save_format='tf')
# print('Saved model to', '/mnt/data/vari_model.h5', 'and', '/mnt/data/vari_saved_model')
#
# # save history
# hist_df = pd.DataFrame(history.history)
# hist_df.to_csv('/mnt/data/vari_model_training_history.csv', index=False)
# print('Saved training history CSV')




# Main training flow - replace paths at top if needed
csv_file = '/home/priyanshu/PycharmProjects/Vegetation_Index_Recommender/Code/image_avg_indices.csv'
preprocessed_dir = '/home/priyanshu/PycharmProjects/Vegetation_Index_Recommender/Code/visualization/outlier'

if not os.path.exists(csv_file) or not os.path.exists(preprocessed_dir):
    raise FileNotFoundError('CSV or preprocessed directory not found. Upload to the paths above or edit the variables in this cell.')

paths, labels = make_dataset_from_csv(csv_file, preprocessed_dir, target_column='avg_VARI')

# splits
n = len(paths)
train_n = int(0.7 * n)
val_n = int(0.15 * n)

np.random.seed(42)
perm = np.random.permutation(n)
train_idx = perm[:train_n]
val_idx = perm[train_n:train_n+val_n]
test_idx = perm[train_n+val_n:]

train_paths, train_labels = paths[train_idx], labels[train_idx]
val_paths, val_labels = paths[val_idx], labels[val_idx]
test_paths, test_labels = paths[test_idx], labels[test_idx]

print('Train/Val/Test sizes:', len(train_paths), len(val_paths), len(test_paths))

# Create tf.data datasets
batch_size = 16
train_ds = make_tf_dataset(train_paths, train_labels, batch_size=batch_size, shuffle=True, augment=True)
val_ds = make_tf_dataset(val_paths, val_labels, batch_size=batch_size, shuffle=False, augment=False)
test_ds = make_tf_dataset(test_paths, test_labels, batch_size=batch_size, shuffle=False, augment=False)

# build model
model = build_model()
model.compile(
    optimizer=optimizers.Adam(learning_rate=5e-4),
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)
model.summary()

# callbacks
es = callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
rlrp = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=8, min_lr=1e-6)

# train
history = model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[es, rlrp])

# evaluate
eval_res = model.evaluate(test_ds)
print('Test eval (loss, rmse):', eval_res)

# ---------------------------
# Save only as pickle
# ---------------------------
pkl_path = '/home/priyanshu/PycharmProjects/Vegetation_Index_Recommender/mnt/data/best_model_vari_pickle.pkl'
model_dict = {
    "model_json": model.to_json(),
    "weights": model.get_weights()
}
joblib.dump(model_dict, pkl_path)
print("Saved pickle model to", pkl_path)

# save training history CSV
hist_path = '/home/priyanshu/PycharmProjects/Vegetation_Index_Recommender/mnt/data/vari_model_training_history.csv'
hist_df = pd.DataFrame(history.history)
hist_df.to_csv(hist_path, index=False)
print('Saved training history CSV')


Found 142 images for target avg_VARI (missing 0)
Train/Val/Test sizes: 99 21 22


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 224, 224,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 112, 112,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        512 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 56, 56,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 28, 28,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 28, 28,    │  1,180,160 │ max_pooling2d_2[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 28,    │      2,048 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │     32,832 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 512)       │     33,280 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 1, 512) │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 28, 28,    │          0 │ batch_normalizat… │
│                     │ 512)              │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ multiply[0][0]  

 Total params: 1,768,769 (6.75 MB)

 Trainable params: 1,766,849 (6.74 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - loss: 283249.0938 - root_mean_squared_error: 532.2115 - val_loss: 0.0095 - val_root_mean_squared_error: 0.0973 - learning_rate: 5.0000e-04
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - loss: 283171.4062 - root_mean_squared_error: 532.1385 - val_loss: 0.0103 - val_root_mean_squared_error: 0.1015 - learning_rate: 5.0000e-04
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - loss: 282870.4062 - root_mean_squared_error: 531.8557 - val_loss: 0.0123 - val_root_mean_squared_error: 0.1109 - learning_rate: 5.0000e-04
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - loss: 282868.4375 - root_mean_squared_error: 531.8538 - val_loss: 0.0052 - val_root_mean_squared_error: 0.0720 - learning_rate: 5.0000e-04
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - loss: 282247.4062 - root_mean_squared_error: 531.2696 - val_loss: 0.2477 - val_root_mean_squared_error: 0.4977 - learning_rate: 5.0000e-04
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - loss

----

**Notes & next steps:**
- If you want a `.pkl` specifically, I can export model weights or the Keras config into a pickle file, but standard TF usage is `.h5` or SavedModel.
- If you want an inference script (Python) to load the saved TF model and predict on a folder of images, I can add that cell or file.
